In [ ]:
from flask import Flask, render_template, Response
import cv2
import numpy as np
from tensorflow.keras.models import load_model

app = Flask(__name__)

# Load the pre-trained CNN model for hand gesture recognition
model = load_model('path/to/your/pretrained_model.h5')  # Replace with the path to your model

# Dictionary to map gesture labels to menu items or commands
gesture_menu_mapping = {
    0: "Pizza",
    1: "Burger",
    2: "Pasta",
    3: "Salad",
    4: "Sandwich",
    5: "Sushi",
    6: "Taco",
    7: "Hot Dog",
    8: "Ice Cream",
    9: "French Fries",
    10: "Chicken Wings",
    11: "Steak",
    12: "Soup",
    13: "Dumplings",
    14: "Noodles",
    15: "Fried Rice",
    16: "Fish and Chips",
    17: "Fried Chicken",
    18: "Shrimp",
    19: "Nachos",
    20: "Cheese",
    21: "Vegetables",
    22: "Fruit",
    23: "Cake",
    24: "Coffee",
    25: "Tea"
}


# Function to preprocess the image and perform prediction
def predict_gesture(image):
    # Preprocess the image (resizing, normalization, etc.)
    # Depending on the model architecture, you may need to resize the image and normalize pixel values
    image = cv2.resize(image, (128, 128))  # Modify the size as per your model input shape
    image = image.astype('float32') / 255.0
    image = np.expand_dims(image, axis=0)

    # Perform prediction using the loaded model
    prediction = model.predict(image)
    gesture_label = np.argmax(prediction)
    return gesture_label

# Function to capture frames from the webcam
def capture_frames():
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gesture_label = predict_gesture(gray)
        gesture = gesture_menu_mapping[gesture_label]
        cv2.putText(frame, gesture, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Convert frame to JPEG format for web display
        ret, buffer = cv2.imencode('.jpg', frame)
        frame_jpg = buffer.tobytes()

        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame_jpg + b'\r\n')

    cap.release()

@app.route('/')
def index():
    return render_template('index.html')  # Create 'index.html' with your web interface design

@app.route('/video_feed')
def video_feed():
    return Response(capture_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    app.run(debug=True)
